## --- Day 16: Packet Decoder ---

Decode the structure of your hexadecimal-encoded BITS transmission; what do you get if you add up the version numbers in all packets?

In [1]:
class PacketReader:
    def __init__(self, hex_str):
        # The bitstring needs to be zero-padded if first bit(s) unset
        self.bitstring = str(bin(int(hex_str, 16)))[2:].zfill(4 * len(hex_str))
        self.index = 0
        self.version_sum = 0

        self.packets = self.read_packets()
    
    def read_next_bits(self, n):
        read = self.bitstring[self.index:self.index + n]
        self.index += n

        return read

    def read_packets(self, num_packets=None, bit_length=None):
        if num_packets is not None:
            # read the next {num_packets} packets
            packets = [self.read_next_packet() for _ in range(num_packets)]
        elif bit_length is not None:
            # Read packets for the next {bit_length} bits
            end_index = self.index + bit_length
            packets = []
            while self.index < end_index:
                packets.append(self.read_next_packet())
        else:
            # Read packet (and all its subpackets)
            packets = [self.read_next_packet()]

        return packets

    def read_next_packet(self):
        packet = {}

        # First 3 bits are packet version, next 3 are packet type
        packet["version"] = int(self.read_next_bits(3), 2)
        packet["type"] = int(self.read_next_bits(3), 2)
        if packet["type"] == 4:
            # Packet contains a literal
            packet["literal"] = self.read_literal()
        else:
            # Packet contains an operator
            packet["length_type"] = self.read_next_bits(1)
            if packet["length_type"] == "0":
                sp_bit_len = int(self.read_next_bits(15), 2)
                packet["sub_packet_bit_len"] = sp_bit_len
                packet["subpackets"] = self.read_packets(bit_length=sp_bit_len)
            else:
                sp_count = int(self.read_next_bits(11), 2)
                packet["sub_packet_count"] = sp_count
                packet["subpackets"] = self.read_packets(num_packets=sp_count)

        self.version_sum += packet["version"]
        
        return packet

    def read_literal(self):
        # First bit of each group of 5 is the continue bit,
        # if continue bit set, read and append the next group.
        literal = 0
        continue_bit = "1"

        while continue_bit == "1":
            group = self.read_next_bits(5)
            literal = (literal << 4) + int(group[1:], 2)
            continue_bit = group[0]

        return literal

    def evaluate(self):
        def _eval(packet):
            ptype = packet["type"]
            if ptype == 4:
                return packet["literal"]
            elif ptype == 0:    # sum
                return sum(_eval(sp) for sp in packet["subpackets"])
            elif ptype == 1:    # product if 2+ subpackets, else the subpacket
                if len(packet["subpackets"]) >= 2:
                    prod = 1
                    for subpacket in packet["subpackets"]:
                        prod *= _eval(subpacket)
                    return prod
                else:
                    return _eval(packet["subpackets"][0])
            elif ptype == 2:    # min
                return min(_eval(sp) for sp in packet["subpackets"])
            elif ptype == 3:    # max
                return max(_eval(sp) for sp in packet["subpackets"])
            elif ptype == 5:    # greater than
                a, b = packet["subpackets"]
                return 1 if _eval(a) > _eval(b) else 0
            elif ptype == 6:    # less than
                a, b = packet["subpackets"]
                return 1 if _eval(a) < _eval(b) else 0
            elif ptype == 7:    # equal to
                a, b = packet["subpackets"]
                return 1 if _eval(a) == _eval(b) else 0
            else:
                raise KeyError(f"Illegal operator type {ptype}")

        return _eval(self.packets[0])


In [2]:
ex1_input = "D2FE28"
ex1_reader = PacketReader(ex1_input)
ex1_packets = ex1_reader.packets

assert ex1_reader.bitstring == '110100101111111000101000'
assert 6 == ex1_packets[0]["version"]
assert 4 == ex1_packets[0]["type"]
assert 2021 == ex1_packets[0]["literal"]

In [3]:
ex1b_reader = PacketReader("38006F45291200")
ex1b_packets = ex1b_reader.packets

assert 1 == ex1b_packets[0]["version"]
assert 6 == ex1b_packets[0]["type"]
assert 27 == ex1b_packets[0]["sub_packet_bit_len"]
assert 2 == len(ex1b_packets[0]["subpackets"])

In [4]:
ex1c_input = "EE00D40C823060"
ex1c_reader = PacketReader(ex1c_input)

ex1c_packets = ex1c_reader.packets

assert 7 == ex1c_packets[0]["version"]
assert 3 == ex1c_packets[0]["type"]
assert 3 == ex1c_packets[0]["sub_packet_count"]
assert 3 == len(ex1c_packets[0]["subpackets"])

In [5]:
# Version Sum examples

assert 16 == PacketReader("8A004A801A8002F478").version_sum
assert 12 == PacketReader("620080001611562C8802118E34").version_sum
assert 23 == PacketReader("C0015000016115A2E0802F182340").version_sum
assert 31 == PacketReader("A0016C880162017C3686B18A3D4780").version_sum


In [6]:
# Part 1 solution
with open("./inputs/Day16.txt") as f:
    p1_input = f.read().strip()

p1_reader = PacketReader(p1_input)

p1_reader.version_sum

917

## --- Part Two ---

Now that you have the structure of your transmission decoded, you can calculate the value of the expression it represents.

Literal values (type ID 4) represent a single number as described above. The remaining type IDs are more interesting:

- Packets with type ID 0 are sum packets - their value is the sum of the values of their sub-packets. If they only have a single sub-packet, their value is the value of the sub-packet.
- Packets with type ID 1 are product packets - their value is the result of multiplying together the values of their sub-packets. If they only have a single sub-packet, their value is the value of the sub-packet.
- Packets with type ID 2 are minimum packets - their value is the minimum of the values of their sub-packets.
- Packets with type ID 3 are maximum packets - their value is the maximum of the values of their sub-packets.
- Packets with type ID 5 are greater than packets - their value is 1 if the value of the first sub-packet is greater than the value of the second sub-packet; otherwise, their value is 0. These packets always have exactly two sub-packets.
- Packets with type ID 6 are less than packets - their value is 1 if the value of the first sub-packet is less than the value of the second sub-packet; otherwise, their value is 0. These packets always have exactly two sub-packets.
- Packets with type ID 7 are equal to packets - their value is 1 if the value of the first sub-packet is equal to the value of the second sub-packet; otherwise, their value is 0. These packets always have exactly two sub-packets.


In [7]:
# Part 2 examples

assert 3 == PacketReader("C200B40A82").evaluate()
assert 54 == PacketReader("04005AC33890").evaluate() # finds the product of 6 and 9, resulting in the value 54.
assert 7 == PacketReader("880086C3E88112").evaluate() # finds the minimum of 7, 8, and 9, resulting in the value 7.
assert 9 == PacketReader("CE00C43D881120").evaluate() # finds the maximum of 7, 8, and 9, resulting in the value 9.
assert 1 == PacketReader("D8005AC2A8F0").evaluate() # produces 1, because 5 is less than 15.
assert 0 == PacketReader("F600BC2D8F").evaluate() # produces 0, because 5 is not greater than 15.
assert 0 == PacketReader("9C005AC2F8F0").evaluate() # produces 0, because 5 is not equal to 15.
assert 1 == PacketReader("9C0141080250320F1802104A08").evaluate() # produces 1, because 1 + 3 = 2 * 2.

In [8]:
# Part 2 solution
p2_reader = PacketReader(p1_input)
p2_reader.evaluate() # == 2532669989335


2536453523344